In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import pickle

In [ ]:
import autoreload
import evotsc
import evotsc_plot
autoreload.reload(evotsc)
autoreload.reload(evotsc_plot)

In [ ]:
## Constants
# Population
nb_indivs = 100
nb_genes = 60

# Genome
intergene = 1000
interaction_dist = 2500
interaction_coef = 0.3
sigma_basal = -0.06
sigma_opt = -0.06
epsilon = 0.03
delta = 1
substeps = 2
default_basal_expression = 0.5
nb_eval_steps = 51

# Environment
sigma_A = 0.1
sigma_B = -0.1

# Mutations
inversion_poisson_lam = 2.0
intergene_poisson_lam = 0.0 #2.0
intergene_mutation_var = 0.0 #1e1
basal_sc_mutation_prob = 0.0 #1e-1
basal_sc_mutation_var = 0.0 #1e-4


In [ ]:
gene_types = ['AB', 'A', 'B'] # Name of each gene type
gene_type_color = ['tab:blue', 'tab:red', 'tab:green'] #AB, A, B

In [ ]:
dpi = 300

In [ ]:
exp_path = pathlib.Path('/Users/theotime/Desktop/evotsc/euler/substep_2_delta_1.0/')

In [ ]:
def get_best_indiv(rep_path, gen):
    
    with open(rep_path.joinpath(f'pop_gen_{gen:06}.evotsc'), 'rb') as save_file:
        pop_rep = pickle.load(save_file)
        
    pop_rep.evaluate()
    
    best_fit = 0
    best_indiv = pop_rep.individuals[0]
    
    try:
        for indiv in pop_rep.individuals:
            if indiv.fitness > best_fit:
                best_fit = indiv.fitness
                best_indiv = indiv
    except AttributeError: # In the neutral control, individuals are not evaluated so there is no fitness field
        pass
    
    return best_indiv

In [ ]:
def plot_expr(indiv, sigma_env, plot_title=None, plot_name=None):
    
    if not indiv.already_evaluated:
        indiv.evaluate(sigma_env, sigma_env)
    
    # Plot only environment A
    temporal_expr = indiv.run_system(sigma_env)

    nb_genes, nb_steps = temporal_expr.shape

    colors = mpl.cm.get_cmap('viridis', nb_genes)(range(nb_genes))

    plt.figure(figsize=(9, 5), dpi=dpi)

    plt.ylim(-0.05, 1.05)

    for gene in range(nb_genes):
        linestyle = 'solid' if indiv.genes[gene].orientation == 0 else 'dashed'
        plt.plot(temporal_expr[indiv.genes[gene].id, :],
                 linestyle=linestyle,
                 color=colors[indiv.genes[gene].id],
                 label=f'Gene {indiv.genes[gene].id}')

    plt.grid(linestyle=':')
    plt.xlabel('Time', fontsize='large')
    plt.ylabel('Expression level', fontsize='large')

    if plot_title:
        plt.title(plot_title)

    plt.tight_layout()
    if plot_name:
        plt.savefig(plot_name, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()


In [ ]:
def plot_genome(indiv, print_ids=False, name=None):

    # Compute gene positions
    gene_pos = np.zeros(len(indiv.genes), dtype=int)
    cur_pos = 0

    for i_gene, gene in enumerate(indiv.genes):
        gene_pos[i_gene] = cur_pos
        cur_pos += gene.intergene
        #print(f'Position gène {i_gene}: {cur_pos}')
    genome_length = cur_pos

    # Plot
    fig, ax = plt.subplots(figsize=(9,9), dpi=dpi)

    rect_width = 0.04
    rect_height = 0.1

    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    circle = plt.Circle(xy=(0, 0), radius=1, linestyle='-', fill=False)
    ax.add_patch(circle)
    ax.set_axis_off()


    #colors = ['tab:blue', 'tab:red', 'tab:green'] # AB: blue, A: red, B: green
    colors = mpl.cm.get_cmap('viridis', nb_genes)(range(nb_genes))
    labels = ['AB', 'A', 'B']

    for i_gene, gene in enumerate(indiv.genes):
        pos_angle = 360 * gene_pos[i_gene] / genome_length
        orient_angle = 360 - pos_angle
        pos_rad = np.radians(pos_angle)
        orient_rad = np.radians(orient_angle)

        ## Plot the gene rectangle

        x0 = (1.0 - rect_height / 2.0) * np.sin(pos_rad)
        y0 = (1.0 - rect_height / 2.0) * np.cos(pos_rad)


        if gene.orientation == 0:
            final_width = rect_width
        else:
            final_width = -rect_width


        rect = plt.Rectangle(xy=(x0, y0),
                             width=final_width,
                             height=rect_height,
                             angle=orient_angle, #in degrees anti-clockwise about xy.
                             facecolor=colors[gene.id],
                             edgecolor='black',
                             label=f'Gene {i_gene}')

        ax.add_patch(rect)

        ## Plot the orientation bar and arrow

        # Bar
        x_lin = (1.0 + (np.array([0.5, 1.0])) * rect_height) * np.sin(pos_rad)
        y_lin = (1.0 + (np.array([0.5, 1.0])) * rect_height) * np.cos(pos_rad)

        plt.plot(x_lin, y_lin, color='black', linewidth=1)

        # Arrow
        dx_arr = rect_width * np.cos(pos_rad) / 3.0
        dy_arr = - rect_width * np.sin(pos_rad) / 3.0

        if gene.orientation == 1: # Reverse
            dx_arr, dy_arr = -dx_arr, -dy_arr

        plt.arrow(x_lin[1], y_lin[1], dx_arr, dy_arr, head_width=0.02, color='black')

        ## Print gene ID
        if print_ids and (i_gene % 5 == 0):
            ha = 'left'
            if gene.orientation == 1:
                ha = 'right'
            plt.text(x=0.92*x0, y=0.92*y0, s=f'{i_gene}', rotation=orient_angle, ha=ha, va='bottom',
                     rotation_mode='anchor')


    ## Legend
    #patches = [mpl.patches.Patch(facecolor=color, edgecolor='black', label=label)
    #           for color, label in zip(colors, labels)]
    #plt.legend(handles=patches, title='Gene type', loc='center')

    line_len = np.pi*indiv.interaction_dist/genome_length
    line_y = -0.3
    plt.plot([-line_len, line_len], [line_y, line_y],
             color='black',
             linewidth=1)
    plt.text(0, line_y - 0.07, 'Gene interaction distance', ha='center')

    if name:
        plt.savefig(name, dpi=300, bbox_inches='tight')

    plt.show()

    plt.close()


In [ ]:
init_genes = evotsc.Gene.generate(intergene=intergene,
                                  nb_genes=nb_genes,
                                  default_basal_expression=default_basal_expression)

In [ ]:
init_indiv = evotsc.Individual(genes=init_genes,
                               interaction_dist=interaction_dist,
                               interaction_coef=interaction_coef,
                               nb_eval_steps=nb_eval_steps,
                               sigma_basal=sigma_basal,
                               sigma_opt=sigma_opt,
                               epsilon=epsilon,
                               delta=delta,
                               substeps=substeps)

In [ ]:
def make_random_indiv(nb_mut):
    genes = evotsc.Gene.generate(intergene=intergene,
                                 nb_genes=nb_genes,
                                 default_basal_expression=default_basal_expression)
    
    indiv = evotsc.Individual(genes=genes,
                              interaction_dist=interaction_dist,
                              interaction_coef=interaction_coef,
                              nb_eval_steps=nb_eval_steps,
                              sigma_basal=sigma_basal,
                              sigma_opt=sigma_opt,
                              epsilon=epsilon,
                              delta=delta,
                              substeps=substeps)
    
    mutation = evotsc.Mutation(basal_sc_mutation_prob=basal_sc_mutation_prob,
                           basal_sc_mutation_var=basal_sc_mutation_var,
                           intergene_poisson_lam=intergene_poisson_lam,
                           intergene_mutation_var=intergene_mutation_var,
                           inversion_poisson_lam=inversion_poisson_lam)
    
    for i_mut in range(nb_mut):
        indiv.mutate(mutation)
    
    return indiv

In [ ]:
mutation = evotsc.Mutation(basal_sc_mutation_prob=basal_sc_mutation_prob,
                           basal_sc_mutation_var=basal_sc_mutation_var,
                           intergene_poisson_lam=intergene_poisson_lam,
                           intergene_mutation_var=intergene_mutation_var,
                           inversion_poisson_lam=inversion_poisson_lam)

In [ ]:
nb_muts = 100

In [ ]:
for i in range(nb_muts):
    init_indiv.mutate(mutation)

In [ ]:
# See how gene activity levels depend on environmental supercoiling
def plot_activity_sigma(indiv, plot_title=None, plot_name=None):

    colors = ['tab:blue', 'tab:red', 'tab:green'] # AB: blue, A: red, B: green
    
    # Initialize the individual
    indiv.evaluate(0.0, 0.0)

    nb_sigmas = 200

    activ = np.zeros((3, nb_sigmas)) # Compute activity for each gene type
    sigmas = np.linspace(-0.2, 0.2, nb_sigmas)

    for i_sigma, sigma_env in enumerate(sigmas):
        # Evaluate the individual in the environment with sigma
        temporal_expr = indiv.run_system(sigma_env)

        # Compute total gene activation levels
        target_steps = 5
        #activ[i_sigma] = np.sum(np.square(temporal_expr[:, indiv.nb_eval_steps-target_steps:])) / (target_steps * indiv.nb_genes)
        
        for i_gene, gene in enumerate(indiv.genes):
            activ[gene.gene_type][i_sigma] += np.sum(np.square(temporal_expr[i_gene, indiv.nb_eval_steps-target_steps:])) / target_steps
            
    activ /= 20 # 20 genes per type

    plt.figure(figsize=(9, 5), dpi=dpi)
    plt.ylabel('Average squared gene activity')
    plt.xlabel('Environment supercoiling')
    plt.ylim(-0.05, 1.05)
    plt.grid(linestyle=':')
    
    if plot_title:
        plt.title(plot_title)

    for i_gene_type, gene_type in enumerate(gene_types):
        plt.plot(sigmas, activ[i_gene_type, :], color=gene_type_color[i_gene_type], label=gene_type)
        
    plt.legend()
    
    
    plt.tight_layout()
        
    if plot_name:
        plt.savefig(plot_name, dpi=dpi, bbox_inches='tight')

In [ ]:
plot_activity_sigma(init_indiv, plot_title='Random individual', plot_name='activ_random')

In [ ]:
for i_rep in range(10):
    indiv = get_best_indiv(exp_path.joinpath(f'rep{i_rep}'), gen=200_000)
    plot_activity_sigma(indiv, plot_title=f'Best rep {i_rep}', plot_name=f'activity_best_rep{i_rep}')

In [ ]:
# Generate N mutants from an individual to see dispersion of gene activity levels
def plot_activity_mutation(indiv, sigma, mutation, plot_name=None):
    
    nb_mut = 1000

    activ = np.zeros(nb_mut)

    for i_mut in range(nb_mut):

        # Generate a new mutant and evaluate it
        mut_indiv = indiv.clone()
        mut_indiv.mutate(mutation)
        (temporal_expr, _), _ = mut_indiv.evaluate(sigma, sigma)

        # Compute total gene activation levels
        target_steps = 5
        activ[i_mut] = np.sum(np.square(temporal_expr[:, mut_indiv.nb_eval_steps-target_steps:])) / (target_steps * mut_indiv.nb_genes)

    # Plot setup    
    plt.figure(figsize=(9, 3), dpi=dpi)
    plt.xlabel('Average squared gene activity')
    #plt.xlim(-0.025, 0.525)
    plt.ylabel('Number of mutants')
    plt.grid(linestyle=':')
    
    # Plot the histogram
    plt.hist(activ)
    
    # Plot the original activity
    (orig_expr, _), _ = indiv.evaluate(sigma, sigma)
    orig_activ = np.sum(np.square(temporal_expr[:, indiv.nb_eval_steps-target_steps:])) / (target_steps * indiv.nb_genes)
    y_min, y_max = plt.ylim()
    plt.vlines(orig_activ, y_min, y_max, linestyle='--', linewidth=1,
                   color='tab:red', label='Original activity level')
    plt.ylim(y_min, y_max)
    
    plt.legend()

    plt.tight_layout()
    
    if plot_name:
        plt.savefig(plot_name, dpi=dpi, bbox_inches='tight')

In [ ]:
for i in range(5):
    plot_activity_mutation(make_random_indiv(nb_mut=100), sigma=sigma_A, mutation=mutation, plot_name=f'robustness_random_{i}')

In [ ]:
plot_activity_mutation(get_best_indiv(exp_path.joinpath('rep0'), gen=200_000), sigma=sigma_A, mutation=mutation)

In [ ]:
plot_activity_mutation(get_best_indiv(exp_path.joinpath('rep0'), gen=200_000), sigma=sigma_B, mutation=mutation)